In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
# !pip install matplotlib numpy pandas tqdm nltk

# for separating ingredients vs non-ingredients
# NOTE: if using Windows to run this, need to download GNU Wget
# !wget -c https://raw.githubusercontent.com/williamLyh/RecipeWithPlans/main/ingredient_set.json -O ingredient_set.json

In [12]:
import os
import re
import string
import numpy as np
import pandas as pd
import random
import json
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR, MultiStepLR
import nltk
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from nltk.translate import meteor

from data import *
from encoder_decoder import *
from train import *
from eval import *

# required for bleu
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /home/junn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

---

In [4]:
SEED = 31989101
HIDDEN_SIZE = 256
MAX_INGR_LEN = 150
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## ensuring reproducibility
def reset_rng():
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

reset_rng()

# to easily read ingredients and instructions
pd.set_option('display.max_colwidth', 2000)

print(f"Using device: {DEVICE}")

Using device: cuda


In [5]:
data_root = "./Cooking_Dataset"

train_df_orig = pd.read_csv(os.path.join(data_root, "train.csv"), usecols=['Ingredients', 'Recipe'])
dev_df_orig = pd.read_csv(os.path.join(data_root, "dev.csv"), usecols=['Ingredients', 'Recipe'])
test_df_orig = pd.read_csv(os.path.join(data_root, "test.csv"), usecols=['Ingredients', 'Recipe'])

In [6]:
train_df = preprocess_data(train_df_orig, max_ingr_len=MAX_INGR_LEN)

Number of data samples before preprocessing: 101340
Number of data samples after preprocessing: 99036 (97.726%)


In [7]:
dev_df = preprocess_data(dev_df_orig, max_ingr_len=MAX_INGR_LEN)

Number of data samples before preprocessing: 797
Number of data samples after preprocessing: 775 (97.240%)


In [8]:
test_df = preprocess_data(test_df_orig, max_ingr_len=MAX_INGR_LEN)

Number of data samples before preprocessing: 778
Number of data samples after preprocessing: 757 (97.301%)


In [9]:
vocab = Vocabulary()
vocab.populate(train_df)
vocab.n_unique_words

100%|██████████| 99036/99036 [00:03<00:00, 28123.66it/s]


44683

In [10]:
recipe_ds = RecipeDataset(train_df, vocab)

In [11]:
encoder = EncoderRNN(vocab.n_unique_words, hidden_size=HIDDEN_SIZE, padding_value=vocab.word2index[PAD_WORD]).to(DEVICE)
# in the training script, decoder is always fed a non-end token and thus never needs to generate padding
decoder = DecoderRNN(hidden_size=HIDDEN_SIZE, output_size=vocab.n_unique_words-1).to(DEVICE)

In [13]:
initial_lr=0.8
min_lr = 0.01
n_epochs = 30
batch_size=128
encoder_optimizer = optim.SGD(encoder.parameters(), lr=initial_lr)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=initial_lr)
# enc_scheduler = CosineAnnealingLR(encoder_optimizer, T_max=n_epochs, eta_min=min_lr)
# dec_scheduler = CosineAnnealingLR(decoder_optimizer, T_max=n_epochs, eta_min=min_lr)
enc_scheduler = MultiStepLR(encoder_optimizer, milestones=[15, 25], gamma=0.5)
dec_scheduler = MultiStepLR(decoder_optimizer, milestones=[15, 25], gamma=0.5)

epoch_losses = train(encoder, decoder, encoder_optimizer, decoder_optimizer, recipe_ds, 
                     n_epochs=n_epochs, vocab=vocab, batch_size=batch_size, 
                     enc_lr_scheduler=enc_scheduler, dec_lr_scheduler=dec_scheduler, 
                     verbose_iter_interval=50)

Starting epoch 1/30, enc lr scheduler: [0.8], dec lr scheduler: [0.8]
(Epoch 0, iter 50/774) Average loss so far: 10.236
(Epoch 0, iter 100/774) Average loss so far: 7.983
(Epoch 0, iter 150/774) Average loss so far: 6.610
(Epoch 0, iter 200/774) Average loss so far: 6.115
(Epoch 0, iter 250/774) Average loss so far: 5.831
(Epoch 0, iter 300/774) Average loss so far: 5.618
(Epoch 0, iter 350/774) Average loss so far: 5.426
(Epoch 0, iter 400/774) Average loss so far: 5.288
(Epoch 0, iter 450/774) Average loss so far: 5.173
(Epoch 0, iter 500/774) Average loss so far: 5.055
(Epoch 0, iter 550/774) Average loss so far: 4.963
(Epoch 0, iter 600/774) Average loss so far: 4.900
(Epoch 0, iter 650/774) Average loss so far: 4.822
(Epoch 0, iter 700/774) Average loss so far: 4.777
(Epoch 0, iter 750/774) Average loss so far: 4.715
Average epoch loss: 5.798
This epoch took 5.165764363606771 mins. Time remaining: 2.0 hrs 29.0 mins.
Starting epoch 2/30, enc lr scheduler: [0.8], dec lr scheduler: 

---

## Metric Sample

In [ ]:
all_ings = get_all_ingredients("./ingredient_set.json")
all_ings_regex = get_ingredients_regex(all_ings)
metric_sample_ings, metric_sample_gold_recipe, metric_sample_generated_recipe = \
    load_metric_sample("./metric_sample.txt")

In [ ]:
prop_inp_ings, n_extra_ings = get_prop_input_num_extra_ingredients(
    metric_sample_ings, metric_sample_generated_recipe, all_ings_regex, verbose=True,
    metric_sample=True)
print(f"\nproportion of input ingredients: {prop_inp_ings}\nnumber of extra ingredients: {n_extra_ings}")

=====Input ingredients in text=====
['orange juice', 'strawberries', 'lemon juice', 'sugar', 'water']

=====All ingredients in text===== 
['vanilla ice cream', 'orange juice', 'strawberries', 'cantaloupe', 'lemon juice', 'sugar', 'water']

proportion of input ingredients: 1.0
number of extra ingredients: 2


In [ ]:
bleu_score = calc_bleu([metric_sample_gold_recipe], [metric_sample_generated_recipe], split_gen=True)
meteor_score = calc_meteor([metric_sample_gold_recipe], [metric_sample_generated_recipe], split_gen=True)
print(f"BLEU score: {bleu_score}, METEOR score: {meteor_score}")

BLEU score: 0.14346607531819988, METEOR score: 0.5736654804270463
